In [1]:
from fastai.tabular import *
import os
import pandas as pd 
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.naive_bayes import ComplementNB
# from sklearn import svm, tree
# import xgboost
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from sklearn.neighbors import KNeighborsClassifier

# from sklearn.linear_model import LogisticRegression
# from sklearn.linear_model import SGDClassifier
# from sklearn.kernel_approximation import RBFSampler


# from sklearn import model_selection
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

# from xgboost import plot_importance
# from xgboost import plot_tree
# import xgboost as xgb

import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
from sklearn.metrics import classification_report


import keras
from keras.models import Sequential
from keras.layers import Dense

import tensorflow.python.util.deprecation as deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
pd.set_option('mode.chained_assignment', None)


Using TensorFlow backend.


In [2]:
path = Path('/Users/bbabu/fastai/DataSamples')

In [3]:
# df_test = pd.read_csv(path/'Test-Group-2.csv', sep=',', header=0)
# df_test_2s = pd.read_csv(path/'Test-Group-2.csv', sep=',', header=0)

In [4]:
# df_test = pd.read_csv(path/'Groups_with_Owners-V3a.csv', sep=';', header=0)
# df_test_2s = pd.read_csv(path/'Groups_with_Owners-V3a.csv', sep=';', header=0)

In [5]:
df_test = pd.read_csv(path/'Groups_with_Owners-V1a.csv', sep=';', header=0)
df_test_2s = pd.read_csv(path/'Groups_with_Owners-V1a.csv', sep=';', header=0)

In [6]:
df_test = df_test.head(77)
df_test_2s = df_test.head(77)

# df_test = df_test.head(77)
# df_test_2s = df_test.head(77)
# df_test_2s.tail()

In [7]:
# df_test.to_csv(path/'Test77.csv', header=True, index=False)

In [8]:
# df_test = pd.read_csv(path/'Test-Group-1.csv', sep=',', header=0)
# df_test_2s = pd.read_csv(path/'Test-Group-1.csv', sep=',', header=0)

In [9]:
# print(df_test)

In [10]:
# df_test.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)
df_test.drop(['MemberDN', 'State'], axis=1, inplace=True)
df_test_2s.drop(['Description', 'MemberDN', 'State', 'Owner'], axis=1, inplace=True)

In [11]:
# df_test.drop(['GroupCountry'], axis=1, inplace=True)
# df_test_2s.drop(['GroupCountry'], axis=1, inplace=True)

In [12]:
# df_test.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)
# df_test_2s.drop(['GroupCountry', 'GroupDomain', 'Domain'], axis=1, inplace=True)

In [13]:
df_test.drop_duplicates(keep='first',inplace=True) 
df_test_2s.drop_duplicates(keep='first',inplace=True) 

In [14]:
df_tests = df_test_2s.groupby(['GroupDN'], as_index=False)

In [16]:
i = 0
for name, df_test_2 in df_tests:
    df_test = df_test_2.copy()
    le = preprocessing.LabelEncoder()
    categorical = list(df_test_2.select_dtypes(include=['object']).columns.values)
    for cat in categorical:
        df_test_2[cat].fillna('NaN', inplace=True)
        df_test_2[cat] = le.fit_transform(df_test_2[cat].astype(str))
    
    filename = path/'Final_Model-NN-V5.sav'
#     filename = path/'Final_Model-NN.sav'
    loaded_model = pickle.load(open(filename, 'rb'))
    
    sc = StandardScaler()    
    X_test = df_test_2
    X_test = sc.fit_transform(X_test)
    pred = loaded_model.predict(X_test)
    if pred.sum() == 0:
        i += 1
    
    final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Title': df_test['Title'], 'Platform': df_test['Platform'], 'City': df_test['City'], 'Owner': pred[:, 0]})
    out = final_df.loc[final_df['Owner'].idxmax()]
    final_df.sort_values(['Owner'], inplace=True, ascending=False)
    final_df.to_csv(path/"".join([(out[0].split(',')[0].replace("CN=","")),'.csv']), header=True, index=False)
    print("Predicated Owner for the group %s is %s with title %s"%((out[0].split(',')[0].replace("CN=","")), out[1], out[3]))




# final_df = pd.DataFrame({'GroupDN': df_test['GroupDN'],'Member': df_test['Member'], 'Manager': df_test['Manager'], 'Owner': pred[:, 0]})
# final_df.to_csv(path/'MLProcessed1-NN-Test.csv', header=True, index=False)
# out = final_df.loc[final_df['Owner'].idxmax()]
# print("Predicated Owner for the group %s is %s with value %s"%(out[0], out[1], out[3]))



Predicated Owner for the group _Disabled msbgsofi001~BgbsofiData~AR~m is d476776 with title CTC Team Manager Collections
Predicated Owner for the group _GIHS Requirements User is a908168 with title Data Center: Wintel System Administrator
Predicated Owner for the group _ROLES_NA_US_SPECIALTY_CRUSH_LEADERS is j116838 with title Depositing/Packaging Op 1 - Hazleton - CCNA - 1st Shift
